In [118]:
import numpy as np
import pandas as pd

In [119]:
names = pd.read_table("VEGPARM.TBL", sep=',\s+',skiprows=[0], skipfooter=288, header=0, engine='python') #Read only the row with the names of the df
names1 = names.squeeze() #Reduce it to an one dimensional array
lista = names1.split() #Separate each str in a cell of the array
lista.append('Index') #Extra cell to match the dimensions of the df

In [120]:
#Read the file ingoring the first 2 rows and setting the index and names of the columns
veg = pd.read_table("VEGPARM.TBL", sep=',\s+', skiprows=[0,1], index_col=18, header=0, usecols=[i for i in range(18)], names=lista, engine='python')

C:\Users\dsp295\AppData\Local\Temp\ipykernel_4524\1789471208.py:2: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  veg = pd.read_table("VEGPARM.TBL", sep=',\s+', skiprows=[0,1], index_col=18, header=0, usecols=[i for i in range(18)], names=lista, engine='python')


In [121]:
#Slicing each table in a different df
USGS_27_1 = veg.iloc[[i for i in range (27)], :]
USGS_27_1 = USGS_27_1.drop("Index",axis=1)#Remove the index column
USGS_27_1

,'SHDFAC,NROOT,RS,RGL,HS,SNUP,MAXALB,LAIMIN,LAIMAX,EMISSMIN,EMISSMAX,ALBEDOMIN,ALBEDOMAX,Z0MIN,Z0MAX,ZTOPV,ZBOTV'
'Urban and Built-Up Land',.10,1.0,200.0,999.0,999.00,0.040,46.0,1.00,1.00,0.88,0.880,0.15,.15,0.5000,0.5000,0.00,0.00
'Dryland Cropland and Pasture',.80,3.0,40.0,100.0,36.25,0.040,66.0,1.56,5.68,0.92,0.985,0.17,.23,0.0500,0.1500,0.50,0.01
'Irrigated Cropland and Pasture',.80,3.0,40.0,100.0,36.25,0.040,66.0,1.56,5.68,0.93,0.985,0.20,.25,0.0200,0.1000,0.50,0.01
'Mixed Dryland/Irrigated Cropland and Pasture',.80,3.0,40.0,100.0,36.25,0.040,66.0,1.00,4.50,0.92,0.985,0.18,.23,0.0500,0.1500,0.50,0.01
'Cropland/Grassland Mosaic',.80,3.0,40.0,100.0,36.25,0.040,68.0,2.29,4.29,0.92,0.980,0.18,.23,0.0500,0.1400,0.50,0.01
'Cropland/Woodland Mosaic',.80,3.0,70.0,65.0,44.14,0.040,60.0,2.00,4.00,0.93,0.985,0.16,.20,0.2000,0.2000,0.50,0.01
'Grassland',.80,3.0,40.0,100.0,36.35,0.040,70.0,0.52,2.90,0.92,0.960,0.19,.23,0.1000,0.1200,0.50,0.01
'Shrubland',.70,3.0,300.0,100.0,42.00,0.030,60.0,0.50,3.66,0.93,0.930,0.25,.30,0.0100,0.0500,0.50,0.10
'Mixed Shrubland/Grassland',.70,3.0,170.0,100.0,39.18,0.035,65.0,0.60,2.60,0.93,0.950,0.22,.30,0.0100,0.0600,0.50,0.10
'Savanna',.50,3.0,70.0,65.0,54.53,0.040,50.0,0.50,3.66,0.92,0.920,0.20,.20,0.1500,0.1500,5.00,0.10


In [126]:
MODIFIED_IGBP_MODIS_NOAH_20_1 = veg.iloc[[i for i in range (66,86)], :]
MODIFIED_IGBP_MODIS_NOAH_20_1 = MODIFIED_IGBP_MODIS_NOAH_20_1.drop("Index",axis=1)#Remove the index column
MODIFIED_IGBP_MODIS_NOAH_20_1

,'SHDFAC,NROOT,RS,RGL,HS,SNUP,MAXALB,LAIMIN,LAIMAX,EMISSMIN,EMISSMAX,ALBEDOMIN,ALBEDOMAX,Z0MIN,Z0MAX,ZTOPV,ZBOTV'
'Evergreen Needleleaf Forest',.70,4.0,125.0,30.0,47.35,0.080,52.0,5.00,6.40,0.95,0.950,0.12,.12,0.5000,0.5000,17.00,8.50
'Evergreen Broadleaf Forest',.95,4.0,150.0,30.0,41.69,0.080,35.0,3.08,6.48,0.95,0.950,0.12,.12,0.5000,0.5000,35.00,1.00
'Deciduous Needleleaf Forest',.70,4.0,150.0,30.0,47.35,0.080,54.0,1.00,5.16,0.93,0.940,0.14,.15,0.5000,0.5000,14.00,7.00
'Deciduous Broadleaf Forest',.80,4.0,100.0,30.0,54.53,0.080,58.0,1.85,3.31,0.93,0.930,0.16,.17,0.5000,0.5000,20.00,11.50
'Mixed Forests',.80,4.0,125.0,30.0,51.93,0.080,53.0,2.80,5.50,0.93,0.970,0.17,.25,0.2000,0.5000,18.00,10.00
'Closed Shrublands',.70,3.0,300.0,100.0,42.00,0.030,60.0,0.50,3.66,0.93,0.930,0.25,.30,0.0100,0.0500,0.50,0.10
'Open Shrublands',.70,3.0,170.0,100.0,39.18,0.035,65.0,0.60,2.60,0.93,0.950,0.22,.30,0.0100,0.0600,0.50,0.10
'Woody Savannas',.70,3.0,300.0,100.0,42.00,0.030,60.0,0.50,3.66,0.93,0.930,0.25,.30,0.0100,0.0500,0.50,0.10
'Savannas',.50,3.0,70.0,65.0,54.53,0.040,50.0,0.50,3.66,0.92,0.920,0.20,.20,0.1500,0.1500,0.50,0.10
'Grasslands',.80,3.0,40.0,100.0,36.35,0.040,70.0,0.52,2.90,0.92,0.960,0.19,.23,0.1000,0.1200,0.50,0.01


In [123]:
NLCD40_40_1 = veg.iloc[[i for i in range (125,165)], :]
NLCD40_40_1 = NLCD40_40_1.drop("Index",axis=1)#Remove the index column
NLCD40_40_1

,'SHDFAC,NROOT,RS,RGL,HS,SNUP,MAXALB,LAIMIN,LAIMAX,EMISSMIN,EMISSMAX,ALBEDOMIN,ALBEDOMAX,Z0MIN,Z0MAX,ZTOPV,ZBOTV'
'Evergreen Needleleaf Forest',.70,4.0,125.0,30.0,47.35,0.080,52.0,5.00,6.40,0.95,0.950,0.12,.12,0.5000,0.5000,17.00,8.50
'Evergreen Broadleaf Forest',.95,4.0,150.0,30.0,41.69,0.080,35.0,3.08,6.48,0.95,0.950,0.12,.12,0.5000,0.5000,35.00,1.00
'Deciduous Needleleaf Forest',.70,4.0,150.0,30.0,47.35,0.080,54.0,1.00,5.16,0.93,0.940,0.14,.15,0.5000,0.5000,14.00,7.00
'Deciduous Broadleaf Forest',.80,4.0,100.0,30.0,54.53,0.080,58.0,1.85,3.31,0.93,0.930,0.16,.17,0.5000,0.5000,20.00,11.50
'Mixed Forest',.80,4.0,125.0,30.0,51.93,0.080,53.0,2.80,5.50,0.93,0.970,0.17,.25,0.2000,0.5000,18.00,10.00
'Closed Shrubland',.70,3.0,300.0,100.0,42.00,0.030,60.0,0.50,3.66,0.93,0.930,0.25,.30,0.0100,0.0500,0.50,0.10
'Open Shrubland',.70,3.0,170.0,100.0,39.18,0.035,65.0,0.60,2.60,0.93,0.950,0.22,.30,0.0100,0.0600,0.50,0.10
'Woody Savanna',.50,3.0,70.0,65.0,54.53,0.040,50.0,0.50,3.66,0.93,0.930,0.25,.30,0.0100,0.0500,0.00,0.00
'Savanna',.50,3.0,70.0,65.0,54.53,0.040,50.0,0.50,3.66,0.92,0.920,0.20,.20,0.1500,0.1500,0.50,0.10
'Grassland',.80,3.0,40.0,100.0,36.35,0.040,70.0,0.52,2.90,0.92,0.960,0.19,.23,0.1000,0.1200,0.50,0.10


In [127]:
USGS_RUC_28_1 = veg.iloc[[i for i in range (204,232)], :]
USGS_RUC_28_1 = USGS_RUC_28_1.drop(["Z0MIN","Z0MAX","ZTOPV","ZBOTV'","Index"], axis=1) #Remove extra columns
#Set the new names of the columns
Names2 = veg.iloc[203]
Names2 = Names2.loc["'SHDFAC"]
Names2 = Names2.split()
Names2.append("")
USGS_RUC_28_1.columns=Names2
#
USGS_RUC_28_1 = USGS_RUC_28_1.set_index(USGS_RUC_28_1.iloc[:, 12]) #Set the new index
USGS_RUC_28_1 = USGS_RUC_28_1.drop("",axis=1)#Remove the index column
USGS_RUC_28_1

,'ALBEDO,Z0,LEMI,PC,SHDFAC,IFOR,RS,RGL,HS,SNUP,LAI,MAXALB'
,,,,,,,,,,,,
'Urban and Built-Up Land',.18,0.5000,0.88,0.40,0.10,9.0,200.0,999.0,999.00,0.040,1.00,40.0
'Dryland Cropland and Pasture',.17,0.2000,0.92,0.30,0.80,7.0,40.0,100.0,36.25,0.040,5.68,64.0
'Irrigated Cropland and Pasture',.18,0.2000,0.92,0.40,0.80,7.0,40.0,100.0,36.25,0.040,5.68,64.0
'Mixed Dryland/Irrigated Cropland and Pasture',.18,0.2000,0.92,0.40,0.80,7.0,40.0,100.0,36.25,0.040,4.50,64.0
'Cropland/Grassland Mosaic',.18,0.2500,0.92,0.40,0.80,7.0,40.0,100.0,36.25,0.040,4.29,64.0
'Cropland/Woodland Mosaic',.16,0.3000,0.93,0.40,0.80,7.0,70.0,65.0,44.14,0.040,4.00,60.0
'Grassland',.19,0.0750,0.92,0.40,0.80,5.0,40.0,100.0,36.35,0.040,2.90,64.0
'Shrubland',.22,0.1000,0.88,0.40,0.70,4.0,300.0,100.0,42.00,0.030,3.66,69.0
'Mixed Shrubland/Grassland',.20,0.1100,0.90,0.40,0.70,4.0,170.0,100.0,39.18,0.035,2.60,67.0


In [128]:
MODI_RUC_21_1 = veg.iloc[[i for i in range (251,272)], :]
MODI_RUC_21_1 = MODI_RUC_21_1.drop(["Z0MIN","Z0MAX","ZTOPV","ZBOTV'","Index"], axis=1) #Remove extra columns
#Set the new names of the columns
Names3 = veg.iloc[250]
Names3 = Names3.loc["'SHDFAC"]
Names3 = Names3.split()
Names3.append("")
MODI_RUC_21_1.columns=Names3
#
MODI_RUC_21_1 = MODI_RUC_21_1.set_index(MODI_RUC_21_1.iloc[:, 12]) #Set the new index
MODI_RUC_21_1 = MODI_RUC_21_1.drop("",axis=1)#Remove the index column
MODI_RUC_21_1

,'ALBEDO,Z0,LEMI,PC,SHDFAC,IFOR,RS,RGL,HS,SNUP,LAI,MAXALB'
,,,,,,,,,,,,
'Evergreen Needleleaf Forest',.12,0.8000,0.950,0.55,0.70,1.0,125.0,30.0,47.35,0.080,6.40,52.0
'Evergreen Broadleaf Forest',.12,0.8000,0.950,0.55,0.95,2.0,150.0,30.0,41.69,0.080,6.48,35.0
'Deciduous Needleleaf Forest',.14,0.8000,0.940,0.55,0.70,4.0,150.0,30.0,47.35,0.080,5.16,54.0
'Deciduous Broadleaf Forest',.16,0.8000,0.930,0.55,0.80,3.0,100.0,30.0,54.53,0.080,3.31,58.0
'Mixed Forests',.13,0.8000,0.940,0.55,0.80,2.0,125.0,30.0,51.93,0.080,5.50,53.0
'Closed Shrublands',.22,0.1000,0.930,0.40,0.70,4.0,300.0,100.0,42.00,0.030,3.66,60.0
'Open Shrublands',.20,0.1000,0.880,0.40,0.70,4.0,170.0,100.0,39.18,0.035,2.60,65.0
'Woody Savannas',.20,0.3000,0.930,0.40,0.70,5.0,300.0,100.0,42.00,0.030,3.66,60.0
'Savannas',.20,0.1500,0.920,0.40,0.50,5.0,70.0,65.0,54.53,0.040,3.66,50.0
